## 1단계: 필수 패키지 설치

In [1]:
!pip install --upgrade azure-ai-evaluation
!pip install pandas numpy matplotlib seaborn

## 2단계: 라이브러리 임포트 및 데이터 로드

In [1]:
import os
import json
import pandas as pd
import numpy as np
from typing import Dict, List, Any
import warnings
warnings.filterwarnings('ignore')

# Microsoft Foundry SDK - GroundednessProEvaluator
try:
    from azure.ai.evaluation import GroundednessProEvaluator
    print("✓ Azure AI Evaluation 임포트 성공")
except ImportError:
    print("⚠ Azure AI Evaluation 설치 필요: pip install azure-ai-evaluation")

# 데이터 로드
with open('data/sample_qa.json', 'r', encoding='utf-8') as f:
    qa_data = json.load(f)

df = pd.DataFrame(qa_data)

print(f"\n✓ 데이터 로드 완료: {len(df)} 개 Q&A 쌍")
print(f"\n에이전트별 분포:")
print(df['agent'].value_counts())

✓ Azure AI Evaluation 임포트 성공

✓ 데이터 로드 완료: 34 개 Q&A 쌍

에이전트별 분포:
agent
product-agent          10
hallucination-agent     7
review-agent            6
policy-agent            6
rag-fail-agent          5
Name: count, dtype: int64


## 3단계: GroundednessProEvaluator 초기화

In [ ]:
# ============================================================
# Azure AI Project 설정 (필수)
# GroundednessProEvaluator는 Azure AI Evaluation Service 사용
# ============================================================

# Azure 인증 임포트
from azure.identity import DefaultAzureCredential, AzureCliCredential

# Azure AI Project 정보 입력
# azure_ai_project = {
#     "subscription_id": "your-subscription-id-here",      # Azure 구독 ID
#     "resource_group_name": "your-resource-group-name",       # 리소스 그룹 이름
#     "project_name": "your-project-name"          # Azure AI 프로젝트 이름
# }

azure_ai_project = "https://{your-project-name}.services.ai.azure.com/api/projects/{your-project-name}"
credential = AzureCliCredential()

# GroundednessProEvaluator 초기화 시도
try:
    groundedness_pro_evaluator = GroundednessProEvaluator(
            azure_ai_project=azure_ai_project,
            credential=credential
    )

except Exception as e:
    print(f"\n⚠ GroundednessProEvaluator 초기화 실패: {str(e)[:200]}")

## 4단계: 전체 데이터에 대한 GroundednessProEvaluator 실행

In [5]:
# 평가를 위한 데이터 준비
groundedness_pro_eval_data = []
for idx, row in df.iterrows():
    groundedness_pro_eval_data.append({
        'query': row['question'],
        'context': row.get('context', row['question']),
        'response': row['response'],
        'agent': row['agent']
    })

print("=== 전체 데이터 GroundednessProEvaluator 평가 시작 ===")
print(f"대상: {len(groundedness_pro_eval_data)} 개 Q&A 쌍\n")

groundedness_pro_results = []

try:
    for i, item in enumerate(groundedness_pro_eval_data):
        try:
            result = groundedness_pro_evaluator(
                response=item['response'],
                context=item['context']
            )
            result['agent'] = item['agent']
            result['context_preview'] = item['context'][:50] + '...' if len(item['context']) > 50 else item['context']
            groundedness_pro_results.append(result)
            
            if (i + 1) % 10 == 0:
                print(f"  [{i + 1}/{len(groundedness_pro_eval_data)}] 완료")
        except Exception as e:
            print(f"  [{i + 1}/{len(groundedness_pro_eval_data)}] 오류: {str(e)[:200]}")

    if groundedness_pro_results:
        groundedness_pro_df = pd.DataFrame(groundedness_pro_results)
        
        # Grounded 비율 계산
        grounded_count = groundedness_pro_df['groundedness_pro_label'].sum() if 'groundedness_pro_label' in groundedness_pro_df.columns else 0
        total_count = len(groundedness_pro_df)
        
        print(f"\n✓ 평가 완료: {len(groundedness_pro_results)}개")
        print(f"Grounded 비율: {grounded_count}/{total_count} ({grounded_count/total_count*100:.1f}%)\n")
        
        # 전체 결과 표시
        print("=== 평가 결과 ===")
        from IPython.display import display
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        display(groundedness_pro_df)
    else:
        print("\n⚠ 평가 결과가 없습니다. Azure AI Project 연결을 확인하세요.")
        
except NameError:
    print("⚠ groundedness_pro_evaluator가 초기화되지 않았습니다.")
    print("Azure AI Project 정보를 설정하고 3단계 셀을 재실행하세요.")

=== 전체 데이터 GroundednessProEvaluator 평가 시작 ===
대상: 34 개 Q&A 쌍

  [10/34] 완료
  [20/34] 완료
  [30/34] 완료

✓ 평가 완료: 34개
Grounded 비율: 23/34 (67.6%)

=== 평가 결과 ===


,groundedness_pro_reason,groundedness_pro_label,groundedness_pro_score,groundedness_pro_threshold,groundedness_pro_result,agent,context_preview
0,All Contents are grounded,True,1,5,pass,product-agent,"갤럭시 S24 사양: 배터리 4,000mAh, 25W 유선 고속 충전, 15W 무선..."
1,All Contents are grounded,True,1,5,pass,product-agent,"노트북 A1: 기본 RAM 16GB DDR5, 최대 32GB 지원, 2개 메모리 슬..."
2,All Contents are grounded,True,1,5,pass,product-agent,"무선 이어폰 X3: 이어버드 8시간, 케이스 포함 28시간, 급속 충전 10분=2시..."
3,All Contents are grounded,True,1,5,pass,product-agent,"스마트워치 W2: IP68 등급, 5ATM 방수, 최대 50m 수심"
4,'태블릿 T5는 큰 화면을 제공하여 동영상 시청과 문서 작업에 적합합니다.' is ...,False,1,5,pass,product-agent,"태블릿 T5: 11인치 AMOLED 디스플레이, 2560x1600 해상도, 120H..."
5,'카메라 C1은 최신 AI 기술을 탑재하여 완벽한 사진을 촬영할 수 있습니다.' i...,False,1,5,pass,product-agent,"카메라 C1: 풀프레임 센서 (36mm x 24mm), 4500만 화소"
6,"'청소 로봇 R7은 스마트폰 앱으로 제어할 수 있으며, 자동으로 충전소로 돌아갑니다...",False,1,5,pass,product-agent,"청소 로봇 R7: 먼지통 600ml, 자동 먼지 배출 스테이션 2.5L"
7,All Contents are grounded,True,1,5,pass,review-agent,"노트북 리뷰 통계: 평균 4.5/5점, 긍정: 속도, 멀티태스킹, 조용함, 부정: 배터리"
8,All Contents are grounded,True,1,5,pass,review-agent,"이어폰 X3 리뷰: 음질 만족도 87%, 저음 우수, 노이즈 캔슬링 우수, 고음 날..."
9,All Contents are grounded,True,1,5,pass,review-agent,"스마트워치 W2 리뷰: 평균 4.2/5점, 배터리 수명 우수, 건강 추적 정확, 앱..."


## 5단계: 할루시네이션 분석

### 할루시네이션 유형
GroundednessProEvaluator는 다음과 같은 할루시네이션을 탐지합니다:

1. **정보 추가 (Fabrication)**: 컨텍스트에 없는 구체적인 수치, 날짜 추가
2. **문맥 왜곡 (Distortion)**: 컨텍스트의 의미를 변경
3. **관련성 부족 (Irrelevance)**: 컨텍스트와 무관한 정보 포함
4. **출처 오류 (Source Error)**: 잘못된 출처 인용

In [6]:
# 할루시네이션 분석 (결과가 있을 경우)
try:
    if groundedness_pro_results:
        print("=== 할루시네이션 분석 ===")
        print(f"\n평가된 샘플: {len(groundedness_pro_results)}개\n")
        
        for i, row in groundedness_pro_df.iterrows():
            is_grounded = row.get('groundedness_pro_label', False)
            agent = row['agent']
            
            if is_grounded:
                status = "✓ 안전 (Grounded)"
            else:
                status = "✗ 고위험 (Hallucination)"
            
            print(f"[{i+1}] {agent}: {status}")
        
        # 요약
        hallucination_count = total_count - grounded_count
        print(f"\n📊 요약:")
        print(f"  - 안전 (Grounded): {grounded_count}개 ({grounded_count/total_count*100:.1f}%)")
        print(f"  - 위험 (Hallucination): {hallucination_count}개 ({hallucination_count/total_count*100:.1f}%)")
    else:
        print("⚠ 평가 결과가 없습니다. Azure AI Project 연결을 확인하세요.")
        
except NameError:
    print("⚠ groundedness_pro_results가 없습니다. 4단계 셀을 먼저 실행하세요.")

=== 할루시네이션 분석 ===

평가된 샘플: 29개

[1] product-agent: ✓ 안전 (Grounded)
[2] product-agent: ✓ 안전 (Grounded)
[3] product-agent: ✓ 안전 (Grounded)
[4] product-agent: ✓ 안전 (Grounded)
[5] product-agent: ✗ 고위험 (Hallucination)
[6] product-agent: ✗ 고위험 (Hallucination)
[7] product-agent: ✗ 고위험 (Hallucination)
[8] review-agent: ✓ 안전 (Grounded)
[9] review-agent: ✓ 안전 (Grounded)
[10] review-agent: ✓ 안전 (Grounded)
[11] review-agent: ✓ 안전 (Grounded)
[12] policy-agent: ✓ 안전 (Grounded)
[13] policy-agent: ✓ 안전 (Grounded)
[14] policy-agent: ✓ 안전 (Grounded)
[15] policy-agent: ✓ 안전 (Grounded)
[16] product-agent: ✓ 안전 (Grounded)
[17] product-agent: ✓ 안전 (Grounded)
[18] product-agent: ✓ 안전 (Grounded)
[19] review-agent: ✓ 안전 (Grounded)
[20] review-agent: ✓ 안전 (Grounded)
[21] policy-agent: ✓ 안전 (Grounded)
[22] policy-agent: ✓ 안전 (Grounded)
[23] hallucination-agent: ✗ 고위험 (Hallucination)
[24] hallucination-agent: ✗ 고위험 (Hallucination)
[25] hallucination-agent: ✗ 고위험 (Hallucination)
[26] hallucination-agent: ✗ 고위험 (Ha

## 7단계: GroundednessEvaluator vs GroundednessProEvaluator 기술적 비교

### 평가 방식
- **GroundednessEvaluator**: Azure OpenAI LLM-judge 기반 평가
- **GroundednessProEvaluator**: Azure AI Evaluation Service의 전문 평가 모델

### 입력 파라미터
- **GroundednessEvaluator**: 
  - 단일 턴: `query`, `response`, `context`
  - 멀티턴: `conversation` (메시지 배열 포함)
  - Azure OpenAI 연결 정보 필요
- **GroundednessProEvaluator**: 
  - 단일 턴: `response`, `context` (query 불필요)
  - 멀티턴: `conversation` (메시지 배열 포함)
  - Azure AI Project 연결 정보 필요

### 출력 형식
- **GroundednessEvaluator**: 
  - `groundedness` (1-5 점수)
  - `gpt_groundedness` (1-5 점수, 일부 버전)
  - 연속적인 점수로 세밀한 구분 가능
- **GroundednessProEvaluator**: 
  - `groundedness_pro_label` (Boolean: True/False)
  - `groundedness_pro_reason` (평가 근거)
  - 이진 분류로 명확한 판단 제공

### 인프라 요구사항
- **GroundednessEvaluator**: 
  - Azure OpenAI Service 배포 필요
  - 모델: GPT-4, GPT-4-turbo 등
- **GroundednessProEvaluator**: 
  - Azure AI Foundry Project 생성 필요
  - 별도 모델 배포 불필요 (관리형 서비스)

### 멀티턴 대화 지원
- **GroundednessEvaluator**: 각 턴을 개별 평가 (수동으로 반복 필요)
- **GroundednessProEvaluator**: 전체 대화를 하나의 단위로 평가 (네이티브 지원)

### 비용 구조
- **GroundednessEvaluator**: Azure OpenAI 토큰 사용량 기반 과금
- **GroundednessProEvaluator**: Azure AI Evaluation Service 트랜잭션 기반 과금

### 응답 시간
- **GroundednessEvaluator**: OpenAI API 호출 지연 시간 (일반적으로 1-3초/요청)
- **GroundednessProEvaluator**: 평가 서비스 처리 시간 (변동 가능)

### 평가 기준
- **GroundednessEvaluator**: LLM의 자연어 이해 기반 평가
- **GroundednessProEvaluator**: 특화된 평가 모델 (할루시네이션 탐지 최적화)